In [2]:
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import re
import random
from random import randint
import time #delay
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import Select
from tqdm import tqdm
 
print(os.getcwd())

c:\Users\maimranj\Desktop\BIMB\backend\legisltaion\puB


In [3]:
driver=webdriver.Edge()

In [4]:
driver=webdriver.Edge()

publication_date = []
pu_no = []
title = []
status_of_legislation = []
related_legislation = []
date_of_commencement = []
pdf_link = []

url = 'https://lom.agc.gov.my/subsid.php?type=pub'
driver.get(url)
time.sleep(random.randint(5, 10))

# Loop through all 813 pages
for page in range(1, 814):
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    pub_list = soup.find('table', attrs={'class': 'table table-striped table-bordered dataTable'}).find_all('tr')[1:]

    for pub in pub_list:
        cols = pub.find_all('td')

        if len(cols) >= 6:
            publication_date.append(cols[0].get_text(strip=True))
            pu_no.append(cols[1].get_text(strip=True))

            title_text = cols[2].get_text(separator="\n", strip=True)
            title.append(title_text)
            
            status_of_legislation.append(cols[3].get_text(strip=True))
            related_legislation.append(cols[4].get_text(strip=True))
            date_of_commencement.append(cols[5].get_text(strip=True))

            pdf_links = cols[6].find_all('a', href=True)
            pdf_link.append(f"https://lom.agc.gov.my{pdf_links[0]['href']}" if pdf_links else 'N/A')

    if page < 813:
        select_element = Select(driver.find_element(By.CSS_SELECTOR, 'div.dataTables_paginate select'))
        select_element.select_by_value(str(page + 1))
        time.sleep(random.randint(5, 10))

pub_df = pd.DataFrame({
    'Publication Date': publication_date,
    'P.U. No': pu_no,
    'Title': title,
    'Status of Legislation': status_of_legislation,
    'Related Legislation': related_legislation,
    'Date of Commencement': date_of_commencement,
    'PDF Link': pdf_link
})

pub_df

,Publication Date,P.U. No,Title,Status of Legislation,Related Legislation,Date of Commencement,PDF Link
0,29/11/2024,P.U. (B) 477/2024,PELANTIKAN ANGGOTA SURUHANJAYA\nAKTA 695 - AKT...,Principal,,-,https://lom.agc.gov.my../../../ilims/upload/po...
1,29/11/2024,P.U. (B) 476/2024,NOTIS KEPADA PIHAK KETIGA\nAKTA 613 - AKTA PEN...,Principal,,-,https://lom.agc.gov.my../../../ilims/upload/po...
2,29/11/2024,P.U. (B) 475/2024,PENETAPAN TARIKH PERMULAAN KUAT KUASA\nAKTA A1...,Principal,,30/11/2024,https://lom.agc.gov.my../../../ilims/upload/po...
3,28/11/2024,P.U. (B) 474/2024,PEMBERITAHUAN NILAI ISIRUNG SAWIT\nDI BAWAH SE...,Principal,,01/12/2024,https://lom.agc.gov.my../../../ilims/upload/po...
4,28/11/2024,P.U. (B) 473/2024,PEMBERITAHUAN NILAI MINYAK PETROLEUM MENTAH D...,Principal,,29/11/2024,https://lom.agc.gov.my../../../ilims/upload/po...
...,...,...,...,...,...,...,...
8123,04/11/1994,P.U. (B) 501/1994,,Principal,,,https://lom.agc.gov.my../../../ilims/upload/po...
8124,24/04/1994,P.U. (B) 197/1994,AKTA BAHASA KEBANGSAAN 1963/67\nNATIONAL LANGU...,Principal,,24/04/1994,https://lom.agc.gov.my../../../ilims/upload/po...
8125,01/10/1971,P.U. (B) 339/1971,Pembetulan Perintah Nafkah (Kemudahan bagi Pen...,Principal,,,N/A
8126,10/04/1969,P.U. (B) 83/1969,Ketetapan menurut Perkara 122a(1)\nAKTA FEDERA...,,,10/04/1969,https://lom.agc.gov.my../../../ilims/upload/po...


In [5]:
driver.close()

In [6]:
pub_df.to_csv('pu(B).csv')